<a href="https://colab.research.google.com/github/RehanShabbir16/Intro_to_NLP_using_Tensorflow/blob/main/imdb_dataset_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow_datasets as tfds

In [ ]:
imdb,info=tfds.load('imdb_reviews',with_info=True,as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.W6TR9O_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.W6TR9O_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.W6TR9O_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
train_data,test_data=imdb['train'],imdb['test']

In [ ]:
training_sentences=[]
training_labels=[]
test_sentences=[]
test_labels=[]
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
for s,l in test_data:
  test_sentences.append(s.numpy().decode('utf8'))
  test_labels.append(l.numpy())

In [ ]:
print(training_sentences[100])
training_labels[100]

I enjoyed this movie,and after watching it,it made me wonder just how many 'Caitlin Rose's' exist in the world.How many other girls have been subjected to this sort of sexual abuse,and torment by classmates and have been too frightened to open their mouth about it? Just how threatening and cruel can teenagers be towards one another,because as this film demonstrates,who's right is not foremost important,its who is popular,and feared which manipulates the minds of youths,and influences them to allow this sort of immorality to happen.Tiffani Amber Thiessen gives a powerful performance as the rape victim,and Brian Austin Green is convincing as the guy torn between the girl he thought he loved,and his best friend.This is the kind of film that doesn't get the exposure it deserves.Remarkable,and brilliant,too good to be just a film made for TV.


1

In [ ]:
training_labels=np.array(training_labels)
test_labels=np.array(test_labels)

In [ ]:
tk=Tokenizer(num_words=10000,oov_token='<OOV>')
tk.fit_on_texts(training_sentences)
word_index=tk.word_index
sequences=tk.texts_to_sequences(training_sentences)
padded=pad_sequences(sequences,maxlen=120,truncating='post')
testing_sequences=tk.texts_to_sequences(test_sentences)
testing_padded=pad_sequences(testing_sequences,maxlen=120)

In [ ]:
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(10000,16,input_length=120),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(padded,training_labels,epochs=10,validation_data=(testing_padded,test_labels))

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.6281 - loss: 0.6087 - val_accuracy: 0.8472 - val_loss: 0.3433
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.8921 - loss: 0.2728 - val_accuracy: 0.8392 - val_loss: 0.3605
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.9635 - loss: 0.1253 - val_accuracy: 0.8308 - val_loss: 0.4198
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9939 - loss: 0.0388 - val_accuracy: 0.8248 - val_loss: 0.5165
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.9979 - loss: 0.0120 - val_accuracy: 0.8194 - val_loss: 0.6180
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.9990 - loss: 0.0057 - val_accuracy: 0.8294 - val_loss: 0.6356
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9999 - loss: 0.0013 - val_accuracy: 0.8186 - val_loss: 0.7384
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9986 - loss: 0.0041 - val_accuracy: 0.

In [ ]:
embedded_layer=model.layers[0]
embedded_weights=embedded_layer.get_weights()[0]
print(embedded_weights.shape)

(10000, 16)


In [ ]:
reverse_word_index=tk.index_word

In [ ]:
import io
out_v=io.open('vecs.tsv','w',encoding='utf-8')
out_m=io.open('meta.tsv','w',encoding='utf-8')

for i in range(1,10000):
  word_name=reverse_word_index[i]
  word_embedding=embedded_weights[i]
  out_m.write(word_name+'\n')
  out_v.write('\t'.join([str(x) for x in word_embedding]))

out_v.close()
out_m.close()

In [ ]:
from google.colab import files
files.download('vecs.tsv')
files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(reverse_word_index.values())

dict_values(['<OOV>', 'the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i', 'this', 'that', 'was', 'as', 'for', 'with', 'movie', 'but', 'film', 'on', 'not', 'you', 'are', 'his', 'have', 'he', 'be', 'one', 'all', 'at', 'by', 'an', 'they', 'who', 'so', 'from', 'like', 'her', 'or', 'just', 'about', "it's", 'out', 'if', 'has', 'some', 'there', 'what', 'good', 'more', 'when', 'very', 'up', 'no', 'time', 'she', 'even', 'my', 'would', 'which', 'only', 'story', 'really', 'see', 'their', 'had', 'can', 'were', 'me', 'well', 'than', 'we', 'much', 'been', 'bad', 'get', 'will', 'do', 'also', 'into', 'people', 'other', 'first', 'great', 'because', 'how', 'him', 'most', "don't", 'made', 'its', 'then', 'way', 'make', 'them', 'too', 'could', 'any', 'movies', 'after', 'think', 'characters', 'watch', 'two', 'films', 'character', 'seen', 'many', 'being', 'life', 'plot', 'never', 'acting', 'little', 'best', 'love', 'over', 'where', 'did', 'show', 'know', 'off', 'ever', 'does', 'better', 'your', 'end'

In [ ]:
!pip install plotly

In [ ]:
embedded_weights[2]


array([-0.01094359, -0.01588303, -0.02673727,  0.0244313 ,  0.03992505,
        0.06113525, -0.10603485, -0.02956065,  0.02195837, -0.03639587,
        0.05128995,  0.04468337, -0.02914023, -0.05349875, -0.00579527,
       -0.16720414], dtype=float32)

In [ ]:
reverse_word_index[2]

'the'

In [ ]:
import plotly.express as px

In [ ]:
import pandas as pd
import plotly.express as px
from sklearn.manifold import TSNE  # For dimensionality reduction

In [ ]:
vecs = pd.DataFrame(embedded_weights)


In [ ]:
meta=pd.DataFrame(reverse_word_index.values())

In [ ]:
vecs.shape

(10000, 16)

In [ ]:
# Assuming meta contains a column named 'label' for visualization
tsne = TSNE(n_components=3,random_state=0, perplexity=16)  # Adjust perplexity as needed
embeddings_2d = tsne.fit_transform(vecs)


KeyboardInterrupt: 

In [ ]:
meta = pd.DataFrame(list(reverse_word_index.values())[:9999])
meta.columns = ['label']  # Assign a column name if needed
meta['x'] = embeddings_2d[1:, 0]
meta['y'] = embeddings_2d[1:, 1]
meta['z'] = embeddings_2d[1:, 2]


In [ ]:
print(meta.columns)
# Assuming labels is a list or Series with the same length as meta


Index([0, 'x', 'y'], dtype='object')


In [ ]:
fig = px.scatter_3d(meta, x='x', y='y',z='z', hover_data=['label'])
fig.show()

In [ ]:
movie_reviews = [
    "This movie was absolutely breathtaking! The cinematography and storytelling were top-notch.",
    "I found the plot to be overly predictable and the characters were poorly developed.",
    "An emotional rollercoaster from start to finish. I was glued to my seat the entire time.",
    "Terribly disappointing. The acting was wooden, and the script was full of clichés.",
    "A fresh and original take on the genre. I loved every minute of it!",
    "The pacing was so slow that I nearly fell asleep halfway through.",
    "A heartwarming tale that left me in tears. The performances were outstanding.",
    "The special effects were great, but the story was a mess and made no sense.",
    "One of the best movies I've seen this year. The direction and soundtrack were phenomenal.",
    "I couldn't wait for it to end. The movie dragged on with no real point or purpose."
]


In [ ]:
pred=tk.texts_to_sequences(movie_reviews)
padded=pad_sequences(pred,maxlen=120)

In [ ]:
rew=['bad','good']

In [ ]:
pred_y=model.predict(padded)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step


In [ ]:
pred_y=np.round(pred_y)
pred_y=pred_y.astype(int)
pred_y=pred_y[0]
pred_y

In [ ]:
for i in pred_y:
  print(i[0])

1
0
1
0
1
0
1
0
1
0


In [ ]:
for i,j in enumerate(pred_y):
  print(movie_reviews[i])
  print(rew[j[0]])

This movie was absolutely breathtaking! The cinematography and storytelling were top-notch.
good
I found the plot to be overly predictable and the characters were poorly developed.
bad
An emotional rollercoaster from start to finish. I was glued to my seat the entire time.
good
Terribly disappointing. The acting was wooden, and the script was full of clichés.
bad
A fresh and original take on the genre. I loved every minute of it!
good
The pacing was so slow that I nearly fell asleep halfway through.
bad
A heartwarming tale that left me in tears. The performances were outstanding.
good
The special effects were great, but the story was a mess and made no sense.
bad
One of the best movies I've seen this year. The direction and soundtrack were phenomenal.
good
I couldn't wait for it to end. The movie dragged on with no real point or purpose.
bad
